<a href="https://colab.research.google.com/github/wardayX/cyhack/blob/main/productname_to_gst_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup & Imports  
Install and import all required libraries.


In [ ]:
!pip install pandas pdfplumber fuzzywuzzy python-Levenshtein sentence-transformers

import pandas as pd
import pdfplumber
from fuzzywuzzy import process as fuzzy_process
from sentence_transformers import SentenceTransformer, util
import torch
import re
from google.colab import files
import io

print("Libraries installed and imported.")

## File Upload Functions  
Define helper functions to upload the HSN PDF and GST CSV.

In [ ]:
def upload_hsn_pdf():
    print("Please upload your HSN Code PDF (SL NO, HS CODE, DESCRIPTION columns).")
    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded.")
        return None, None
    file_name = list(uploaded.keys())[0]
    print(f"Uploaded '{file_name}'")
    return file_name, uploaded[file_name]

def upload_gst_csv():
    print("\nPlease upload your GST Rates CSV.")
    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded.")
        return None, None
    file_name = list(uploaded.keys())[0]
    print(f"Uploaded '{file_name}'")
    return file_name, uploaded[file_name]

print("File upload functions defined.")

## Parse HSN PDF with Aggregate Hierarchical Descriptions
Extract HS codes and descriptions from the uploaded PDF with enriching parent HSN codes by concatenating child descriptions.


In [ ]:
def parse_hsn_pdf(pdf_content):
    data = []
    try:
        with pdfplumber.open(io.BytesIO(pdf_content)) as pdf:
            for i, page in enumerate(pdf.pages):
                print(f"Processing PDF page {i+1}/{len(pdf.pages)}...")
                tables = page.extract_tables()
                if tables:
                    for table in tables:
                        header = table[0]
                        if header and 'HS CODE' in str(header).upper() and 'DESCRIPTION' in str(header).upper():
                            data_rows = table[1:]
                        else:
                            data_rows = table

                        for row in data_rows:
                            if len(row) >= 3:
                                sl_no, hs_code, description = row[0], row[1], row[2]
                                hs_code = str(hs_code).replace('\n', ' ').strip() if hs_code else None
                                description = str(description).replace('\n', ' ').strip() if description else None
                                if hs_code and description:
                                    data.append({'HS_Code_PDF': hs_code, 'Description_PDF': description})
                    continue

                text = page.extract_text()
                if text:
                    lines = text.split('\n')
                    for line in lines:
                        match_hs = re.search(r'^\s*(\d{4,8})\s+(.+)', line)
                        if match_hs:
                            hs_code = match_hs.group(1).strip()
                            description = match_hs.group(2).strip()
                            description = re.sub(r'\s{2,}', ' ', description)
                            if hs_code and description:
                                data.append({'HS_Code_PDF': hs_code, 'Description_PDF': description})
                        else:
                            print(f"Could not parse line: {line}")


        if not data:
            print("Warning: No data extracted from PDF. PDF parsing might need custom logic for your file format.")
            print("Consider using page.extract_text() and custom regex if tables are not well-structured.")
            return pd.DataFrame(columns=['HS_Code_PDF', 'Description_PDF'])

        df_hsn = pd.DataFrame(data)
        df_hsn['HS_Code_PDF'] = df_hsn['HS_Code_PDF'].astype(str).str.replace(r'\W+', '', regex=True).str.strip().str.lower()
        df_hsn.dropna(subset=['HS_Code_PDF'], inplace=True)
        df_hsn.drop_duplicates(subset=['HS_Code_PDF'], keep='first', inplace=True)
        print(f"Extracted {len(df_hsn)} unique HSN entries from PDF.")
        return df_hsn
    except Exception as e:
        print(f"Error parsing PDF: {e}")
        print("Please ensure the PDF is not scanned (image-based) and has extractable text.")
        return pd.DataFrame(columns=['HS_Code_PDF', 'Description_PDF'])

print("PDF parsing function defined.")

def aggregate_hsn_descriptions(df_hsn_input):
    if df_hsn_input.empty:
        print("HSN PDF data is empty. Skipping aggregation.")
        return df_hsn_input.copy()

    print("\nAggregating hierarchical HSN descriptions from PDF data...")
    df_hsn = df_hsn_input.copy()
    df_hsn['HS_Code_PDF'] = df_hsn['HS_Code_PDF'].astype(str)
    df_hsn.sort_values(by='HS_Code_PDF', inplace=True)
    df_hsn.reset_index(drop=True, inplace=True)

    aggregated_descriptions = {}
    unique_hs_codes = sorted(df_hsn['HS_Code_PDF'].unique())

    for parent_hs in unique_hs_codes:
        parent_row = df_hsn[df_hsn['HS_Code_PDF'] == parent_hs]
        if parent_row.empty or pd.isna(parent_row['Description_PDF'].iloc[0]):
            continue

        current_descriptions = [parent_row['Description_PDF'].iloc[0]]
        for child_hs in unique_hs_codes:
            if child_hs.startswith(parent_hs) and len(child_hs) > len(parent_hs):
                child_row = df_hsn[df_hsn['HS_Code_PDF'] == child_hs]
                if not child_row.empty and pd.notna(child_row['Description_PDF'].iloc[0]):
                    current_descriptions.append(child_row['Description_PDF'].iloc[0])
        aggregated_descriptions[parent_hs] = ". ".join(list(dict.fromkeys(current_descriptions)))

    df_hsn['Aggregated_Description_PDF'] = df_hsn['HS_Code_PDF'].map(aggregated_descriptions)
    df_hsn['Aggregated_Description_PDF'].fillna(df_hsn['Description_PDF'], inplace=True)

    print("HSN description aggregation complete.")
    return df_hsn

## Parse GST Rates CSV  
Read the GST CSV with fallback encodings, clean and normalize columns.


In [1]:
def parse_gst_csv(csv_content):
    df_gst = None
    encodings_to_try = ['utf-8', 'latin1', 'cp1252', 'iso-8859-1']

    for encoding in encodings_to_try:
        try:
            csv_file_like_object = io.BytesIO(csv_content)
            df_gst = pd.read_csv(csv_file_like_object, encoding=encoding)
            print(f"GST CSV loaded successfully with encoding: {encoding}")
            break
        except UnicodeDecodeError:
            print(f"Failed to decode CSV with encoding: {encoding}. Trying next...")
        except Exception as e:
            print(f"Error reading CSV with encoding {encoding}: {e}")
            df_gst = None

    if df_gst is None:
        print("Error: Could not read or decode the GST CSV file with common encodings.")
        print("Please ensure your CSV file is saved in a compatible format (e.g., UTF-8, Latin-1, CP1252) or specify the correct encoding.")
        return pd.DataFrame()
    print("GST CSV loaded. Original columns found:", df_gst.columns.tolist())

    column_mapping = {
        'Chapter/Heading/Sub-heading/Tariffitem': 'HS_Code_GST',
        'DescriptionofGoods': 'Description_GST',
        'CGST(%)': 'CGST_Rate',
        'SGST/UTGST(%)': 'SGST_Rate',
        'IGST(%)': 'IGST_Rate',
        'CompensationCess': 'Compensation_Cess_Raw'
    }

    actual_mapping_to_rename = {}
    for csv_header_pattern, internal_name in column_mapping.items():
        for actual_csv_column_name in df_gst.columns:
            if csv_header_pattern.lower().strip() == actual_csv_column_name.lower().strip():
                actual_mapping_to_rename[actual_csv_column_name] = internal_name
                break

    if actual_mapping_to_rename:
        df_gst.rename(columns=actual_mapping_to_rename, inplace=True)
        print("Columns after initial renaming attempt:", df_gst.columns.tolist())
    else:
        print("No column renaming mappings were applied based on column_mapping.")

    required_internal_cols = ['HS_Code_GST', 'Description_GST', 'CGST_Rate', 'SGST_Rate', 'IGST_Rate']

    if 'Compensation_Cess_Raw' not in df_gst.columns:
        found_cess_col = None
        for col_name in df_gst.columns:
            if "compensationcess" == col_name.lower().strip() and col_name not in actual_mapping_to_rename.values():
                found_cess_col = col_name
                break
            elif "cess" in col_name.lower() and "compensation" in col_name.lower() and col_name not in actual_mapping_to_rename.values():
                found_cess_col = col_name
                break
        if found_cess_col:
            df_gst.rename(columns={found_cess_col: 'Compensation_Cess_Raw'}, inplace=True)
            print(f"Renamed '{found_cess_col}' to 'Compensation_Cess_Raw'.")
        else:
            print("Warning: 'Compensation_Cess_Raw' (or a mappable equivalent) not found. Assuming no compensation cess for now.")
            df_gst['Compensation_Cess_Raw'] = "Nil"

    required_internal_cols.append('Compensation_Cess_Raw')

    missing_cols = [col for col in required_internal_cols if col not in df_gst.columns]
    if missing_cols:
        print(f"Error: Missing expected internal columns after all renaming attempts: {missing_cols}")
        print(f"Current available columns in DataFrame: {df_gst.columns.tolist()}")
        print("Please ensure your `column_mapping` in Cell 4 correctly maps your CSV headers to the expected internal names.")
        return pd.DataFrame()

    df_gst = df_gst[required_internal_cols].copy()
    df_gst['HS_Code_GST'] = df_gst['HS_Code_GST'].astype(str).str.replace(r'\W+', '', regex=True).str.strip().str.lower()
    rate_cols_internal = ['CGST_Rate', 'SGST_Rate', 'IGST_Rate']
    for col in rate_cols_internal:
        df_gst[col] = df_gst[col].astype(str).str.replace('%', '').str.strip()
        df_gst[col] = pd.to_numeric(df_gst[col], errors='coerce').fillna(0)

    def parse_cess(value):
        value_str = str(value).lower().strip()
        if not value_str or value_str in ['no', 'false', 'nil', 'exempt', 'exempted', '0', '0%', '0.0', '0.0%','-']:
            return 0.0, False
        match_rate = re.search(r'(\d+\.?\d*)', value_str)
        if match_rate:
            try:
                rate = float(match_rate.group(1))
                return rate, True
            except ValueError:
                pass
        if pd.notna(value) and value_str not in ['no', 'false', 'nil', 'exempt', 'exempted', '0', '0%', '0.0', '0.0%','-']:
            return 0.0, True
        return 0.0, False

    cess_parsed = df_gst['Compensation_Cess_Raw'].apply(parse_cess)
    df_gst['Compensation_Cess_Rate'] = cess_parsed.apply(lambda x: x[0])
    df_gst['Is_Compensation_Cess'] = cess_parsed.apply(lambda x: x[1])
    df_gst.drop(columns=['Compensation_Cess_Raw'], inplace=True)

    df_gst['Is_Exempted'] = (
        (df_gst['CGST_Rate'] == 0) &
        (df_gst['SGST_Rate'] == 0) &
        (df_gst['IGST_Rate'] == 0) &
        ((~df_gst['Is_Compensation_Cess']) | (df_gst['Compensation_Cess_Rate'] == 0))
    )

    df_gst.dropna(subset=['HS_Code_GST'], inplace=True)
    df_gst.drop_duplicates(subset=['HS_Code_GST'], keep='first', inplace=True)
    print(f"Processed {len(df_gst)} unique HSN entries from GST CSV.")
    print("Final columns in GST DataFrame after processing:", df_gst.columns.tolist())
    return df_gst

print("GST CSV parsing function (CORRECTED RENAMING LOGIC + ENCODING HANDLING) defined.")

GST CSV parsing function (CORRECTED RENAMING LOGIC + ENCODING HANDLING) defined.


## Load & Parse Inputs  
1. Upload HSN PDF → parse → aggregate  
2. Upload GST CSV → parse  


In [ ]:
hsn_pdf_name, hsn_pdf_content = upload_hsn_pdf()
df_hsn_pdf = pd.DataFrame()
if hsn_pdf_content:
    df_hsn_pdf = parse_hsn_pdf(hsn_pdf_content)
    if not df_hsn_pdf.empty:
        print("\n--- Parsed HSN PDF Data (Sample BEFORE Aggregation) ---")
        print(df_hsn_pdf.head())
        df_hsn_pdf_aggregated = aggregate_hsn_descriptions(df_hsn_pdf)
    else:
        df_hsn_pdf_aggregated = pd.DataFrame()
else:
    df_hsn_pdf_aggregated = pd.DataFrame()
    print("\nHSN PDF content not available. Skipping HSN PDF processing and aggregation.")

gst_csv_name, gst_csv_content = upload_gst_csv()
df_gst_rates = pd.DataFrame()
if gst_csv_content:
    df_gst_rates = parse_gst_csv(gst_csv_content)

gst_csv_name, gst_csv_content = upload_gst_csv()
df_gst_rates = pd.DataFrame()
if gst_csv_content:
    df_gst_rates = parse_gst_csv(gst_csv_content)
    if not df_gst_rates.empty:
        print("\n--- Parsed GST Rates CSV Data (Sample) ---")
        print(df_gst_rates.head())
        print("\nGST Data Columns:", df_gst_rates.columns)

if df_hsn_pdf.empty and df_gst_rates.empty:
    print("\nERROR: Neither HSN PDF nor GST CSV data could be loaded. Cannot proceed.")
elif df_gst_rates.empty:
    print("\nERROR: GST CSV data could not be loaded. Tax rates are essential. Cannot proceed.")